**Assignment 3**

**Name: Sai Naga Sanjana Chippada**

**ID: 017546776**

**English to French**

In [1]:
!pip install torch==2.0.0 torchtext==0.15.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 839.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 86.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 110.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 96.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install torchdata==0.6.0

In [3]:
!pip install portalocker>=2.0.0

In [4]:
!pip install numpy==1.23.5

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 50.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imbalanced-learn 0.13.0 requires numpy<3,>=1.24.3, but you have numpy 1.23.5 which is incompatible.
albumentations 2.0.4 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
chex 0.1.88 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.
torchvision 0.20.1+cu124 requires torch==2.5.1, but you have torch 2.0.0 which is incompatible.
pymc 5.20.1 requires numpy>=1.25.0, but you have numpy 1.23.5 which is incompatible.
jax 0.4.33 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
xarray 2025.1.2 requires numpy>=1.24, but you have nu

In [1]:
!python -m spacy download fr_core_news_sm
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 64.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 107.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
import torch
import spacy
from torch.utils.data import DataLoader, Dataset
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from typing import Iterable, List
import math
import torch.nn as nn
from torch.nn import Transformer
from torch import Tensor
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence


In [3]:
# Load spaCy tokenizers for English and French
spacy_eng = spacy.load("en_core_web_sm")
spacy_fr = spacy.load("fr_core_news_sm")

In [4]:
# Set device configuration (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [5]:
# Tokenization function
def tokenize_english(text):
    return [token.text.lower() for token in spacy_eng.tokenizer(text)]
def tokenize_french(text):
    return [token.text.lower() for token in spacy_fr.tokenizer(text)]

english_text_file = "english.txt"
french_text_file = "french.txt"


In [6]:
with open(english_text_file, "r", encoding="utf-8") as f:
    english_sentences = f.read().splitlines()

with open(french_text_file, "r", encoding="utf-8") as f:
    french_sentences = f.read().splitlines()

In [7]:
# Build vocabulary
def yield_tokens(sentences, tokenizer):
    for sentence in sentences:
        yield tokenizer(sentence)

vocab_en = build_vocab_from_iterator(yield_tokens(english_sentences, tokenize_english), specials=["<unk>", "<pad>", "<bos>", "<eos>"])
vocab_fr = build_vocab_from_iterator(yield_tokens(french_sentences, tokenize_french), specials=["<unk>", "<pad>", "<bos>", "<eos>"])

vocab_en.set_default_index(vocab_en["<unk>"])
vocab_fr.set_default_index(vocab_fr["<unk>"])

In [8]:
# Convert sentences to tensor
def numericalize(sentence, vocab, tokenizer):
    return [vocab["<bos>"]] + [vocab[token] for token in tokenizer(sentence)] + [vocab["<eos>"]]

In [9]:
# Custom dataset class for translation
class TranslationDataset(Dataset):
    def __init__(self, src_sentences, tgt_sentences, src_vocab, tgt_vocab, src_tokenizer, tgt_tokenizer):
        self.src_sentences = src_sentences
        self.tgt_sentences = tgt_sentences
        self.src_vocab = src_vocab
        self.tgt_vocab = tgt_vocab
        self.src_tokenizer = src_tokenizer
        self.tgt_tokenizer = tgt_tokenizer

    def __len__(self):
        return len(self.src_sentences)

    def __getitem__(self, index):
        src_tensor = torch.tensor(numericalize(self.src_sentences[index], self.src_vocab, self.src_tokenizer))
        tgt_tensor = torch.tensor(numericalize(self.tgt_sentences[index], self.tgt_vocab, self.tgt_tokenizer))
        return src_tensor, tgt_tensor

In [10]:
# Collate function for padding sequences
def collate_batch_translation(batch):
    src_batch, tgt_batch = zip(*batch)
    src_batch = pad_sequence(src_batch, padding_value=vocab_en["<pad>"], batch_first=True)  # Use English vocab for source padding
    tgt_batch = pad_sequence(tgt_batch, padding_value=vocab_fr["<pad>"], batch_first=True)  # Use French vocab for target padding
    return src_batch.to(device), tgt_batch.to(device)  # Move tensors to the specified device


In [11]:
# DataLoader
BATCH_SIZE = 32
translation_dataset = TranslationDataset(english_sentences, french_sentences, vocab_en, vocab_fr, tokenize_english, tokenize_french)
translation_loader = DataLoader(translation_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch_translation)


In [12]:
# Transformer Model Components - Positional Encoding
class PositionalEncoding(nn.Module):  # No fixed sequence length
    def __init__(self, model_dim, dropout_prob=0.1, max_tokens=5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout_prob)
        self.encoding_matrix = torch.zeros(max_tokens, model_dim)
        token_positions = torch.arange(0, max_tokens, dtype=torch.float).unsqueeze(1)
        scale_factor = torch.exp(torch.arange(0, model_dim, 2).float() * (-math.log(10000.0) / model_dim))
        self.encoding_matrix[:, 0::2] = torch.sin(token_positions * scale_factor)
        self.encoding_matrix[:, 1::2] = torch.cos(token_positions * scale_factor)
        self.encoding_matrix = self.encoding_matrix.unsqueeze(0)

    def forward(self, input_tensor):
        input_tensor = input_tensor + self.encoding_matrix[:, :input_tensor.size(1)].to(input_tensor.device)  # Align device and size
        return self.dropout(input_tensor)


In [13]:
# Token Embedding
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_capacity, embed_dim):
        super(TokenEmbedding, self).__init__()
        self.embed_layer = nn.Embedding(vocab_capacity, embed_dim)

    def forward(self, input_tokens):
        return self.embed_layer(input_tokens.long()) * math.sqrt(self.embed_layer.embedding_dim)


In [14]:
class TransformerModel(nn.Module):
    def __init__(self, input_vocab_size, output_vocab_size, model_dim=512, num_heads=8, num_encoder_layers=6, num_decoder_layers=6, dim_feedforward=2048, dropout=0.1):
        super().__init__()
        self.model_dim = model_dim
        self.input_embedding = nn.Embedding(input_vocab_size, model_dim)
        self.output_embedding = nn.Embedding(output_vocab_size, model_dim)
        self.position_encoder = PositionalEncoding(model_dim, dropout)

        # Pass correct argument names to nn.Transformer
        self.transformer = nn.Transformer(
            d_model=model_dim,
            nhead=num_heads,
            num_encoder_layers=num_encoder_layers,
            num_decoder_layers=num_decoder_layers,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            batch_first=True
        )

        self.final_layer = nn.Linear(model_dim, output_vocab_size)
        self.dropout_layer = nn.Dropout(dropout)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Set device
        self.initialize_weights()

    def initialize_weights(self):
        weight_range = 0.1
        self.input_embedding.weight.data.uniform_(-weight_range, weight_range)
        self.output_embedding.weight.data.uniform_(-weight_range, weight_range)
        self.final_layer.bias.data.zero_()
        self.final_layer.weight.data.uniform_(-weight_range, weight_range)

    def generate_square_subsequent_mask(self, size):
        mask = (torch.triu(torch.ones((size, size), device=self.device)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def create_mask(self, src_seq, tgt_seq):
        src_len = src_seq.shape[1]
        tgt_len = tgt_seq.shape[1]

        tgt_mask = self.generate_square_subsequent_mask(tgt_len).to(self.device)  # Ensure mask is on the correct device
        src_mask = torch.zeros((src_len, src_len), device=self.device).type(torch.bool)

        src_pad_mask = (src_seq == vocab_en["<pad>"])  # src padding mask
        tgt_pad_mask = (tgt_seq == vocab_fr["<pad>"])  # tgt padding mask
        return src_mask, tgt_mask, src_pad_mask, tgt_pad_mask

    def forward(self, src_seq, tgt_seq):
        src_mask, tgt_mask, src_pad_mask, tgt_pad_mask = self.create_mask(src_seq, tgt_seq)

        # Ensure src_seq and tgt_seq are on the correct device
        src_embeddings = self.position_encoder(self.input_embedding(src_seq) * math.sqrt(self.model_dim)).to(self.device)
        tgt_embeddings = self.position_encoder(self.output_embedding(tgt_seq) * math.sqrt(self.model_dim)).to(self.device)

        transformer_out = self.transformer(src_embeddings, tgt_embeddings, src_mask=src_mask, tgt_mask=tgt_mask, src_key_padding_mask=src_pad_mask, tgt_key_padding_mask=tgt_pad_mask)

        return self.final_layer(transformer_out)

    def encode(self, src_seq):
        src_embeddings = self.position_encoder(self.input_embedding(src_seq) * math.sqrt(self.model_dim)).to(self.device)
        src_mask = torch.zeros((src_seq.shape[1], src_seq.shape[1]), device=self.device).type(torch.bool)
        return self.transformer.encoder(src_embeddings, src_mask)

    def decode(self, tgt_seq, encoded_memory):
        tgt_embeddings = self.position_encoder(self.output_embedding(tgt_seq) * math.sqrt(self.model_dim)).to(self.device)
        tgt_mask = self.generate_square_subsequent_mask(tgt_seq.shape[1]).to(self.device)
        return self.transformer.decoder(tgt_embeddings, encoded_memory, tgt_mask)


In [15]:
# Initialize model parameters
INPUT_DIM = len(vocab_en)
OUTPUT_DIM = len(vocab_fr)
EMB_SIZE = 256
NHEAD = 8
FFN_HID_DIM = 256
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3
DROPOUT = 0.1

# Instantiate the Transformer model with correct parameter names
model = TransformerModel(
    INPUT_DIM,
    OUTPUT_DIM,
    model_dim=EMB_SIZE,
    num_heads=NHEAD,
    num_encoder_layers=NUM_ENCODER_LAYERS,
    num_decoder_layers=NUM_DECODER_LAYERS,
    dim_feedforward=FFN_HID_DIM,
    dropout=DROPOUT
).to(device)

# Print the model architecture to verify the setup
print(model)


TransformerModel(
  (input_embedding): Embedding(13757, 256)
  (output_embedding): Embedding(23814, 256)
  (position_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0-2): 3 x TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
          )
          (linear1): Linear(in_features=256, out_features=256, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=256, out_features=256, bias=True)
          (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
      )
      (norm): LayerNorm((256,), eps=1e-05, 

In [16]:
criterion = nn.CrossEntropyLoss(ignore_index=vocab_fr["<pad>"])
optimizer = optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)


In [17]:
def train_epoch(model, translation_loader, criterion, optimizer):
    model.train()
    total_loss = 0.
    for src, tgt in translation_loader:
        src, tgt = src.to(device), tgt.to(device)  # Ensure data is moved to the correct device
        optimizer.zero_grad()
        tgt_input = tgt[:, :-1]
        output = model(src, tgt_input)  # Teacher forcing, pass source and target (except last token)
        loss = criterion(output.reshape(-1, output.shape[-1]), tgt[:, 1:].reshape(-1))  # Compare with target (except first token)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)  # Gradient clipping
        optimizer.step()
        total_loss += loss.item()

    return total_loss / len(translation_loader)


In [18]:
def train(model, translation_loader, criterion, optimizer, epochs=3):
    for epoch in range(1, epochs+1):
        epoch_start_time = time.time()
        train_loss = train_epoch(model, translation_loader, criterion, optimizer)
        val_loss = evaluate(model, translation_loader, criterion)
        print(f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, Epoch time = {(time.time() - epoch_start_time):.3f}s")


In [19]:
def evaluate(model, translation_loader, criterion):
    model.eval()
    total_loss = 0.
    with torch.no_grad():
        for src, tgt in translation_loader:
            src, tgt = src.to(device), tgt.to(device)  # Ensure data is on the correct device
            tgt_input = tgt[:, :-1]
            output = model(src, tgt_input)
            loss = criterion(output.reshape(-1, output.shape[-1]), tgt[:, 1:].reshape(-1))
            total_loss += loss.item()
    return total_loss / len(translation_loader)


In [20]:
def greedy_decode(model, src_tensor, src_vocab, tgt_vocab, max_len=50):

    model.eval()

    # Encode the source sentence
    with torch.no_grad():
        memory = model.encode(src_tensor)

    # Initialize the target sentence with <bos>
    tgt_indices = [tgt_vocab["<bos>"]]

    for _ in range(max_len):
        tgt_tensor = torch.tensor(tgt_indices).unsqueeze(0).to(model.device)  # (1, seq_len)

        # Decode using the current sequence
        with torch.no_grad():
            output = model.decode(tgt_tensor, memory)
            logits = model.final_layer(output[:, -1, :])  # Get logits for the last token

            # Get the highest probability token
            next_token = logits.argmax(dim=1).item()

        tgt_indices.append(next_token)

        # Stop at <eos>
        if next_token == tgt_vocab["<eos>"]:
            break

    # Convert token indices to words
    decoded_sentence = " ".join([tgt_vocab.get_itos()[idx] for idx in tgt_indices[1:]])  # Skip <bos>

    return decoded_sentence


In [21]:
def translate_english_to_french(sentence):
    # Tokenize the input sentence and convert it to indices (numericalize)
    src_tokens = numericalize(sentence, vocab_en, tokenize_english)
    src_tensor = torch.tensor(src_tokens).unsqueeze(0).to(device)  # Add batch dimension and move to device
    output_tokens = model(src_tensor, src_tensor).argmax(dim=-1)
    return " ".join(vocab_fr.get_itos()[idx] for idx in output_tokens[0])


In [22]:
import time
# Start training
EPOCHS = 10
train(model, translation_loader, criterion, optimizer, epochs=EPOCHS)

/usr/local/lib/python3.11/dist-packages/torch/nn/functional.py:4999: UserWarning: Support for mismatched src_key_padding_mask and src_mask is deprecated. Use same type for both instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/functional.py:4999: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:544: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at ../aten/src/ATen/native/transformers/attention.cpp:150.)
  return torch._transformer_encoder_layer_fwd(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/activation.py:1144: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at ../

Epoch: 1, Train loss: 3.617, Val loss: 2.536, Epoch time = 232.695s
Epoch: 2, Train loss: 2.465, Val loss: 1.963, Epoch time = 230.597s
Epoch: 3, Train loss: 2.065, Val loss: 1.696, Epoch time = 232.686s
Epoch: 4, Train loss: 1.848, Val loss: 1.537, Epoch time = 232.891s
Epoch: 5, Train loss: 1.709, Val loss: 1.428, Epoch time = 230.976s
Epoch: 6, Train loss: 1.607, Val loss: 1.349, Epoch time = 245.676s
Epoch: 7, Train loss: 1.528, Val loss: 1.272, Epoch time = 245.255s
Epoch: 8, Train loss: 1.461, Val loss: 1.217, Epoch time = 233.773s
Epoch: 9, Train loss: 1.410, Val loss: 1.182, Epoch time = 230.468s
Epoch: 10, Train loss: 1.394, Val loss: 1.185, Epoch time = 228.344s


In [36]:
# English to French Conversion
def translate_english_to_french(sentence, model, vocab_en, vocab_fr, tokenize_english):
    return greedy_decode(model, torch.tensor(numericalize(sentence, vocab_en, tokenize_english)).unsqueeze(0).to(model.device), vocab_en, vocab_fr)

# Example French sentence
sentence_en = "how was the movie yesterday ?"

# Translate French to English using the trained model
translated_sentence = translate_english_to_french(sentence_en, model, vocab_en, vocab_fr, tokenize_english)

# Print the translated sentence
print("Original English Sentence:", sentence_en)
print("Translated French Sentence:", translated_sentence)

Original English Sentence: how was the movie yesterday ?
Translated French Sentence: comment était le film hier   ? <eos>


In [26]:
# English to French Conversion
def translate_english_to_french(sentence, model, vocab_en, vocab_fr, tokenize_english):
    return greedy_decode(model, torch.tensor(numericalize(sentence, vocab_en, tokenize_english)).unsqueeze(0).to(model.device), vocab_en, vocab_fr)

# Example French sentence
sentence_en = "She loves chocolate."

# Translate French to English using the trained model
translated_sentence = translate_english_to_french(sentence_en, model, vocab_en, vocab_fr, tokenize_english)

# Print the translated sentence
print("Original English Sentence:", sentence_en)
print("Translated French Sentence:", translated_sentence)

Original English Sentence: She loves chocolate.
Translated French Sentence: elle adore le chocolat . <eos>


**French to English**

In [27]:
#Initialize model

INPUT_DIM = len(vocab_fr)  # Source is French
OUTPUT_DIM = len(vocab_en)  # Target is English
EMB_SIZE = 256
NHEAD = 8
FFN_HID_DIM = 256
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3
model_french_en = TransformerModel(
    INPUT_DIM,
    OUTPUT_DIM,
    model_dim=EMB_SIZE,  # Corrected from d_model to model_dim
    num_heads=NHEAD,  # Corrected from nhead to num_heads
    num_encoder_layers=NUM_ENCODER_LAYERS,  # Number of encoder layers
    num_decoder_layers=NUM_DECODER_LAYERS,  # Number of decoder layers
    dim_feedforward=FFN_HID_DIM,  # Feedforward layer dimension
    dropout=DROPOUT  # Dropout for regularization
).to(device)
print(model_french_en)

# Training setup
criterion = nn.CrossEntropyLoss(ignore_index=vocab_en["<pad>"])
optimizer = optim.Adam(model_french_en.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

TransformerModel(
  (input_embedding): Embedding(23814, 256)
  (output_embedding): Embedding(13757, 256)
  (position_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0-2): 3 x TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
          )
          (linear1): Linear(in_features=256, out_features=256, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=256, out_features=256, bias=True)
          (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
      )
      (norm): LayerNorm((256,), eps=1e-05, 

In [28]:
# DataLoader
BATCH_SIZE = 64
translation_dataset = TranslationDataset(french_sentences, english_sentences, vocab_fr, vocab_en, tokenize_french, tokenize_english)
translation_loader = DataLoader(translation_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch_translation)



In [29]:
# Training function
def train_epoch(model, translation_loader, criterion, optimizer):
    model.train()
    total_loss = 0.
    for src, tgt in translation_loader:
        optimizer.zero_grad()
        tgt_input = tgt[:, :-1]
        output = model(src, tgt_input)  # Teacher forcing, pass source and target (except last token)
        loss = criterion(output.reshape(-1, output.shape[-1]), tgt[:, 1:].reshape(-1))  # Compare with target (except first token)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)  # Gradient clipping
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(translation_loader)

In [30]:
def train(model, translation_loader, criterion, optimizer, epochs=3):
    for epoch in range(1, epochs+1):
        epoch_start_time = time.time()
        train_loss = train_epoch(model, translation_loader, criterion, optimizer)
        val_loss = evaluate(model, translation_loader, criterion)
        print(f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(time.time() - epoch_start_time):.3f}s")

In [31]:
def evaluate(model, translation_loader, criterion):
    model.eval()
    total_loss = 0.
    with torch.no_grad():
        for src, tgt in translation_loader:
            tgt_input = tgt[:, :-1]
            output = model(src, tgt_input)
            loss = criterion(output.reshape(-1, output.shape[-1]), tgt[:, 1:].reshape(-1))
            total_loss += loss.item()
    return total_loss / len(translation_loader)

In [33]:
import torch
import time

EPOCHS = 10
# Train the model for French to English
train(model_french_en, translation_loader, criterion, optimizer, epochs=EPOCHS)


def greedy_decode_fr_to_en(model, src_tensor, src_vocab, tgt_vocab, max_len=50):

    model.eval()

    # Encode the French source sentence
    with torch.no_grad():
        memory = model.encode(src_tensor)

    # Initialize the target sentence with <bos>
    tgt_indices = [tgt_vocab["<bos>"]]

    for _ in range(max_len):
        # Convert the target sequence into a tensor
        tgt_tensor = torch.tensor(tgt_indices).unsqueeze(0).to(model.device)  # (1, seq_len)

        # Decode using current target sequence
        with torch.no_grad():
            output = model.decode(tgt_tensor, memory)
            logits = model.final_layer(output[:, -1, :])  # Get logits for the last token
            next_token = logits.argmax(dim=1).item()  # Get the highest probability token

        tgt_indices.append(next_token)

        # Stop decoding if <eos> token is generated
        if next_token == tgt_vocab["<eos>"]:
            break

    # Convert token indices to words
    decoded_sentence = " ".join([tgt_vocab.get_itos()[idx] for idx in tgt_indices[1:]])  # Skip <bos>

    return decoded_sentence

def translate_french_to_english(sentence, model, vocab_fr, vocab_en, tokenize_fr, max_len=50):

    # Tokenize and numericalize the input French sentence
    src_tokens = numericalize(sentence, vocab_fr, tokenize_fr)
    src_tensor = torch.tensor(src_tokens).unsqueeze(0).to(model.device)  # (1, seq_len)

    # Use greedy decoding to generate the English translation
    return greedy_decode_fr_to_en(model, src_tensor, vocab_fr, vocab_en, max_len)


Epoch: 1, Train loss: 3.239, Val loss: 2.422, Epoch time = 143.425s
Epoch: 2, Train loss: 2.338, Val loss: 1.825, Epoch time = 136.818s
Epoch: 3, Train loss: 1.901, Val loss: 1.506, Epoch time = 134.293s
Epoch: 4, Train loss: 1.640, Val loss: 1.307, Epoch time = 134.268s
Epoch: 5, Train loss: 1.463, Val loss: 1.164, Epoch time = 134.344s
Epoch: 6, Train loss: 1.334, Val loss: 1.057, Epoch time = 135.002s
Epoch: 7, Train loss: 1.235, Val loss: 0.978, Epoch time = 139.765s
Epoch: 8, Train loss: 1.156, Val loss: 0.909, Epoch time = 138.565s
Epoch: 9, Train loss: 1.090, Val loss: 0.852, Epoch time = 137.031s
Epoch: 10, Train loss: 1.034, Val loss: 0.802, Epoch time = 136.420s


In [37]:
# Example French sentence
sentence_fr = "comment était le film hier   ?"

# Translate French to English using the trained model
translated_sentence = translate_french_to_english(sentence_fr, model_french_en, vocab_fr, vocab_en, tokenize_french)

# Print the translated sentence
print("Original French Sentence:", sentence_fr)
print("Translated English Sentence:", translated_sentence)

Original French Sentence: comment était le film hier   ?
Translated English Sentence: how was the movie yesterday ? <eos>


In [35]:
# Example French sentence
sentence_fr = "elle adore le chocolat ."

# Translate French to English using the trained model
translated_sentence = translate_french_to_english(sentence_fr, model_french_en, vocab_fr, vocab_en, tokenize_french)

# Print the translated sentence
print("Original French Sentence:", sentence_fr)
print("Translated English Sentence:", translated_sentence)

Original French Sentence: elle adore le chocolat .
Translated English Sentence: she loves chocolate . <eos>


**Comparison of English-to-French and French-to-English Translation Models**

Two separate sequence-to-sequence (Seq2Seq) models were built:

**English-to-French Model** – Translates sentences from English to French.
**French-to-English Model** – Translates sentences from French to English.

To evaluate performance, the output of the English-to-French model was used as input for the French-to-English model. The objective was to determine whether the back-translated sentence matched the original English sentence.

**Test Case 1**

**Original Sentence (English):** How was the movie yesterday?

**Translated Sentence (French):** Comment était le film hier ?

**Back-Translated Sentence (English):** How was the movie yesterday?

The French-to-English model successfully reproduced the original sentence without any loss of meaning.

**Test Case 2**

**Original Sentence (English):** She loves chocolate.

**Translated Sentence (French):** Elle adore le chocolat.

**Back-Translated Sentence (English):** She loves chocolate.

Both test cases demonstrate that the translations remain accurate and consistent when converting from English to French and back to English.

**Analysis of Model Performance**
The results indicate that both models effectively learn the mapping between English and French, preserving meaning and grammatical structure. The ability to reconstruct the original sentence suggests that the models have achieved a high level of translation quality.


In [38]:
pip install nltk


In [39]:
from nltk.translate.bleu_score import sentence_bleu

# Reference translations (actual translations)
reference_1 = ["comment était le film hier ?".split()]
reference_2 = ["elle adore le chocolat .".split()]

# Candidate translations (model-generated translations)
candidate_1 = "comment était le film hier ?".split()
candidate_2 = "elle adore le chocolat .".split()

# Compute BLEU scores
bleu_1 = sentence_bleu(reference_1, candidate_1)
bleu_2 = sentence_bleu(reference_2, candidate_2)

print(f"BLEU Score for Sentence 1: {bleu_1:.4f}")
print(f"BLEU Score for Sentence 2: {bleu_2:.4f}")


BLEU Score for Sentence 1: 1.0000
BLEU Score for Sentence 2: 1.0000


In [46]:
from rouge_score import rouge_scorer

# Define the reference and candidate translations
reference = "how was the movie yesterday"
candidate = "how was the film yesterday"

# Initialize the ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Compute the ROUGE scores
scores = scorer.score(reference, candidate)

# Print the ROUGE scores
print("ROUGE-1: ", scores['rouge1'])
print("ROUGE-2: ", scores['rouge2'])
print("ROUGE-L: ", scores['rougeL'])


ROUGE-1:  Score(precision=0.8, recall=0.8, fmeasure=0.8000000000000002)
ROUGE-2:  Score(precision=0.5, recall=0.5, fmeasure=0.5)
ROUGE-L:  Score(precision=0.8, recall=0.8, fmeasure=0.8000000000000002)


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=a603aa7aa38bfe821a84772321b679c3c9e5dde64353208bf5386778db902b83
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score
